# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,277.83,68,45,5.38,TF,1707415481
1,1,bredasdorp,-34.5322,20.0403,292.97,64,88,5.25,ZA,1707415481
2,2,arenapolis,-14.4503,-56.8461,300.73,72,89,3.60,BR,1707415481
3,3,khodzha-maston,38.7446,68.6270,274.83,50,100,5.26,TJ,1707415481
4,4,hermanus,-34.4187,19.2345,293.04,73,81,4.26,ZA,1707415362


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=800,frame_height=800,scale=2,size="Humidity",color="City")
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 300) & (city_data_df["Max Temp"] > 290) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df["Cloudiness"] < 10)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,santiago de cao,-7.9589,-79.2392,299.22,78,0,4.12,PE,1707415483
36,36,geraldton,-28.7667,114.6000,299.38,22,3,2.57,AU,1707415484
45,45,east london,-33.0153,27.9116,295.67,73,0,1.54,ZA,1707415484
59,59,marsa alam,25.0757,34.8918,293.56,65,0,3.13,EG,1707415486
63,63,al kharijah,25.4514,30.5464,290.97,29,0,3.98,EG,1707415486


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,santiago de cao,PE,-7.9589,-79.2392,78,
36,geraldton,AU,-28.7667,114.6000,22,
45,east london,ZA,-33.0153,27.9116,73,
59,marsa alam,EG,25.0757,34.8918,65,
63,al kharijah,EG,25.4514,30.5464,29,
67,illizi,DZ,26.4833,8.4667,18,
135,reggane,DZ,26.7158,0.1714,13,
192,tocopilla,CL,-22.0920,-70.1979,70,
215,mukalla,YE,14.5425,49.1242,66,
265,brandon,US,27.9378,-82.2859,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params={
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santiago de cao - nearest hotel: Hostal El Encanto
geraldton - nearest hotel: Ocean Centre Hotel
east london - nearest hotel: No hotel found
marsa alam - nearest hotel: Blue House Hotel
al kharijah - nearest hotel: Kharga Hotel
illizi - nearest hotel: بوناقة
reggane - nearest hotel: No hotel found
tocopilla - nearest hotel: hotel Isidora
mukalla - nearest hotel: فندق الجابري
brandon - nearest hotel: Brandon Motor Lodge
samalut - nearest hotel: No hotel found
mashi - nearest hotel: No hotel found
nkhata bay - nearest hotel: Cairo Motel
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
springbok - nearest hotel: Elkoweru Guesthouse
tamanrasset - nearest hotel: فندق الأمان
ataq - nearest hotel: الشارقة بلازا
medenine - nearest hotel: No hotel found
port pirie - nearest hotel: Ellen Hotel
zastron - nearest hotel: No hotel found
ratnagiri - nearest hotel: Vihar
luwingu - nearest hotel: No hotel found
san pablo villa de mitla - nearest hotel: Don Aji
kigoma - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
24,santiago de cao,PE,-7.9589,-79.2392,78,Hostal El Encanto
36,geraldton,AU,-28.7667,114.6000,22,Ocean Centre Hotel
45,east london,ZA,-33.0153,27.9116,73,No hotel found
59,marsa alam,EG,25.0757,34.8918,65,Blue House Hotel
63,al kharijah,EG,25.4514,30.5464,29,Kharga Hotel
67,illizi,DZ,26.4833,8.4667,18,بوناقة
135,reggane,DZ,26.7158,0.1714,13,No hotel found
192,tocopilla,CL,-22.0920,-70.1979,70,hotel Isidora
215,mukalla,YE,14.5425,49.1242,66,فندق الجابري
265,brandon,US,27.9378,-82.2859,62,Brandon Motor Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hover_plot = hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=800,frame_height=800,scale=2,hover_cols=["Hotel Name","Country"])

# Display the map
hover_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)